In [78]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math 
import string
import re
from sklearn.pipeline import Pipeline
from nltk.tokenize import wordpunct_tokenize
import nltk
from transformers import AutoTokenizer
from datasets import Dataset

In [2]:
lines = list()

with open("train.txt", "r") as fp:
    lines = fp.readlines()

In [3]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
y_train = [x.lower().split('\t')[0] for x in lines]
x_train = [''.join(x.lower().split('\t')[1:]) for x in lines]
x_train = [wordpunct_tokenize(x) for x in x_train]

In [5]:
labels = set(y_train)

In [6]:
model_checkpoint = "bert-base-uncased"
batch_size = 8

In [7]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

{'deceptivenegative',
 'deceptivepositive',
 'truthfulnegative',
 'truthfulpositive'}

In [9]:
id2label

{0: 'deceptivenegative',
 1: 'truthfulnegative',
 2: 'truthfulpositive',
 3: 'deceptivepositive'}

In [10]:
df_all = pd.DataFrame([' '.join(x) for x in x_train], columns=['text'])

In [11]:
yt = pd.DataFrame(y_train)
yt_np = np.array(y_train)

In [41]:
idx = np.random.permutation(df_all.index)
df_all = df_all.reindex(idx)
y_train = yt.reindex(idx)

df_train = df_all.iloc[:1000]
df_val = df_all.iloc[1000:]

y_val = y_train.iloc[1000:]
y_train = y_train.iloc[:1000]


df_train.head()

,text
1280,great hotel !! the staff is very friendly and ...
1069,the omni chicago hotel is not pretty as they a...
198,after reading so many great reviews i booked t...
239,the hilton chicago was amazing !! it is close ...
1294,my first trip back to chicago in about 7 years...


In [42]:
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
  

In [61]:
#df_all.iloc[[1,2,3:7]]

SyntaxError: invalid syntax (859811017.py, line 1)

In [70]:
idx = np.random.permutation(df_all.index)
df_all = df_all.reindex(idx)
y_train = yt.reindex(idx)

x_trains = list()
y_trains = list()

N = 5

for var in range(N):
    len_fold = 1400/N
    x_trains.append((df_all.iloc[var*int(len_fold):(var+1)*int(len_fold)], df_all.iloc[np.r_[0:int(var*len_fold) , int((var+1)*len_fold):1400]] ))
    y_trains.append((y_train.iloc[var*int(len_fold):(var+1)*int(len_fold)], y_train.iloc[np.r_[0:int(var*len_fold), int((var+1)*len_fold):1400]] ))

#x_trains = list(divide_chunks(df_all, 280))
#y_trains = list(divide_chunks(y_train, 280))

In [77]:
y_trains[3]

(                      0
 166   deceptivenegative
 641    truthfulpositive
 1286   truthfulnegative
 337   deceptivepositive
 800   deceptivepositive
 ...                 ...
 925    truthfulnegative
 1034  deceptivenegative
 1135  deceptivepositive
 1318  deceptivenegative
 1240  deceptivenegative
 
 [280 rows x 1 columns],
                       0
 967   deceptivenegative
 1327  deceptivenegative
 932   deceptivepositive
 1343  deceptivepositive
 1374   truthfulnegative
 ...                 ...
 1360  deceptivenegative
 540    truthfulpositive
 1128  deceptivenegative
 178   deceptivepositive
 354   deceptivenegative
 
 [1120 rows x 1 columns])

df_aux = df_train.copy()
curr_size = 1400
for el in df_aux.iterrows():
    print(el[0])
    length = 128
    while length + 256 < len(el[1]['text'].split(' ')):
        tex = el[1]['text'].split(' ')[length:length+256]
        #print(el[1]['text'])
        #print('len', len(el[1]['text'].split(' ')))
        #print(df_train.loc[el[0], 'text'])
        df_train.loc[el[0], 'text'] = ' '.join(el[1]['text'].split(' ')[:256])
        #print(df_train.loc[el[0], 'text'])
        new_df = pd.DataFrame(pd.Series({'text': ' '.join(tex)}, name = curr_size))
        #print(new_df.transpose())
        df_train = pd.concat([df_train, new_df.transpose()], ignore_index=False)
        new_df = pd.DataFrame(pd.Series(y_train.loc[el[0]], name = curr_size))
        print(new_df)#.transpose())
        y_train = pd.concat([y_train, new_df.transpose()], ignore_index=False)
        curr_size += 1
        length += 128
    

In [15]:
_dct_train = df_train.to_dict()
_dct_train['text'] = list(_dct_train['text'].values())
_dct_train['label'] = y_train.apply(lambda x: label2id[x[0]], axis=1)

_dataset_train =  Dataset.from_dict(_dct_train)

_dct_val = df_val.to_dict()
_dct_val['text'] = list(_dct_val['text'].values())
_dct_val['label'] = y_val.apply(lambda x: label2id[x[0]], axis=1)


_dataset_val =  Dataset.from_dict(_dct_val)

_dataset_dct = dict()
_dataset_dct['train'] = _dataset_train
_dataset_dct['val'] = _dataset_val


In [16]:
_dataset_train[2]

{'text': 'the fairmont chicago millenium park hotel provides a rich , beautiful , cultured atmosphere . during my stay , i felt like royalty . with unparalleled design schemes and beautiful , tranquil views , the mood during my entire stay was one of bliss . the guest services department was second - to - none . the staff throughout the hotel were obviously concerned with pleasing the customer and making my stay as wonderful as possible . when dealing with a staff member at fairmont , i felt as if i were the only guest in the place . the food and drink selections were vast and fit for a king . i will definitely recommend fairmont chicago millenium park to my friends and colleagues . i will also greatly look forward to my next stay .',
 'label': 3}

In [17]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [18]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

In [19]:
tokenized_train = _dataset_train.map(preprocess_function, batched=True)
tokenized_val = _dataset_val.map(preprocess_function, batched=True)


Map: 100%|███████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 9044.47 examples/s]


In [20]:
#tokenized_val[0]

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=256)

In [22]:
#!pip install evaluate

In [23]:
import evaluate

accuracy = evaluate.load("accuracy")

In [24]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels_ = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels_)

In [25]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy'
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.626619,0.735000
2,No log,0.405002,0.850000
3,No log,0.526671,0.845000
4,0.432900,0.520177,0.880000
5,0.432900,0.737396,0.857500
6,0.432900,0.576676,0.885000
7,0.432900,0.651320,0.882500
8,0.022100,0.705824,0.872500
9,0.022100,0.706277,0.872500
10,0.022100,0.705441,0.872500


TrainOutput(global_step=1330, training_loss=0.17134488316855037, metrics={'train_runtime': 1491.9589, 'train_samples_per_second': 7.118, 'train_steps_per_second': 0.891, 'total_flos': 1397144792309760.0, 'train_loss': 0.17134488316855037, 'epoch': 10.0})

In [27]:
trainer.model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [28]:
trainer.evaluate()

{'eval_loss': 0.5766761302947998,
 'eval_accuracy': 0.885,
 'eval_runtime': 17.8059,
 'eval_samples_per_second': 22.464,
 'eval_steps_per_second': 2.808,
 'epoch': 10.0}

In [29]:
trainer.save_model("./bert_005")

In [30]:
#model = trainer.model

In [31]:
text = "Bad. Super bad. Very bad hotel. worst hotel ever. I have everyone"

In [32]:
test_set = list()

with open("test_just_reviews.txt", "r") as fp:
    test_set = fp.readlines()

In [33]:
test_set = [' '.join(x.split(' ')[:512]) for x in test_set]

In [34]:
tokenizer = AutoTokenizer.from_pretrained("bert_005/")

tokenizer(text, return_tensors="pt").to(device="cuda:0")

{'input_ids': tensor([[ 101, 2919, 1012, 3565, 2919, 1012, 2200, 2919, 3309, 1012, 5409, 3309,
         2412, 1012, 1045, 2031, 3071,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [35]:
tokenizer = AutoTokenizer.from_pretrained("bert_005/")
tokenized_test = list()

for rev in test_set:
    inputs = tokenizer(rev, return_tensors="pt", truncation=True, max_length=256).to(device="cuda:0")
    tokenized_test.append(inputs)
#inputs.get_device()

In [36]:
results = list()

In [37]:
#tokenized_test

In [38]:
with torch.no_grad():
    for rev in tokenized_test:
        logits = model(**rev).logits
        predicted_class_id = logits.argmax().item()
        results.append(model.config.id2label[predicted_class_id])

In [39]:
results

['deceptivepositive',
 'deceptivenegative',
 'deceptivepositive',
 'truthfulnegative',
 'truthfulpositive',
 'truthfulnegative',
 'deceptivepositive',
 'deceptivepositive',
 'truthfulnegative',
 'deceptivenegative',
 'deceptivepositive',
 'deceptivepositive',
 'truthfulnegative',
 'truthfulnegative',
 'truthfulpositive',
 'deceptivepositive',
 'deceptivepositive',
 'truthfulpositive',
 'deceptivenegative',
 'truthfulpositive',
 'deceptivepositive',
 'deceptivepositive',
 'deceptivepositive',
 'deceptivenegative',
 'deceptivepositive',
 'deceptivenegative',
 'truthfulpositive',
 'truthfulpositive',
 'truthfulnegative',
 'deceptivenegative',
 'truthfulpositive',
 'deceptivepositive',
 'deceptivepositive',
 'deceptivenegative',
 'truthfulnegative',
 'deceptivenegative',
 'truthfulpositive',
 'deceptivepositive',
 'truthfulnegative',
 'deceptivenegative',
 'truthfulpositive',
 'truthfulnegative',
 'deceptivenegative',
 'truthfulpositive',
 'deceptivenegative',
 'truthfulnegative',
 'decept

In [40]:
pd.Series(results).to_csv("Results_005.txt")